## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [2]:
import pandas as pd
from IPython.display import display

In [3]:
data = pd.read_csv('/datasets/data.csv')
display(data.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

In [4]:
display(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


None

"Общий трудовой стаж в днях" "days_employed" представлено в формате float

In [5]:
#для всех пустых значений в поле 'стаж' выводим значения поля 'оклад'
display('Пропуски в поле трудовой стаж соответствуют пропускам в поле ежемесячный оклад\n',data[data['days_employed'].isnull()]['total_income'].unique())
display('Количество пустых строк в полях трудовой стаж и оклад',data[data['days_employed'].isnull()]['dob_years'].count())

'Пропуски в поле трудовой стаж соответствуют пропускам в поле ежемесячный оклад\n'

array([nan])

'Количество пустых строк в полях трудовой стаж и оклад'

2174

В ходе анализа обнаружены пропуски у одних и тех же клиентов в графах "общий трудовой стаж в днях" и "ежемесячный доход" это примерно 10% от общего набора данных. Возможно эти клиенты получают "серую" зарплату и не хотят предоставлять оклад и стаж работы.

In [6]:
display('Количество клиентов со отрицательным стажем',
      data[data['days_employed'] < 0]['days_employed'].count())
display('Количество клиентов с положительным стажем оказались только:',
      data[data['days_employed'] > 0]['income_type'].unique())

'Количество клиентов со отрицательным стажем'

15906

'Количество клиентов с положительным стажем оказались только:'

array(['пенсионер', 'безработный'], dtype=object)

Выявлено, что положительные значения  в данных о стаже присутствуют только у пенсионеров и безработных, что скорее всего свидетельствует о разных источниках для датасета.

In [7]:
display('Уникальные значения в поле образование:',data['education'].unique())
display('Уникальные значения в поле семейное полжение:',data['family_status'].unique())

'Уникальные значения в поле образование:'

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

'Уникальные значения в поле семейное полжение:'

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

"Образование" и "Семейное положение" указано разными регистрами, а также существуют дубликаты.

In [8]:
display(data['children'].unique())
data['children'] = data['children'].replace(-1,1) # не может быть минус 1 ребенок, ошибка

array([ 1,  0,  3,  2, -1,  4, 20,  5])

В поле "Количество детей" есть индекс с отрицательным занчением. Заменим "-1" на "1"

In [9]:
display(data['purpose'].unique())

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Поле "Цель получения кредита" содержит дубликаты целей.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [10]:
import numpy as np

In [11]:
data['days_employed'] = data['days_employed'].fillna(0)

mediana = data.groupby('income_type')['total_income'].median().to_dict()

def fill_income(row):
    if np.isnan(row['total_income']):       
        return mediana[row["income_type"]]
    else:
        return row["total_income"]
    
data['total_income'] = data.apply(fill_income, axis=1)
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


### Вывод

В графе "общий трудовой стаж в днях" выявлены значения с "-" (15 906), так как эти данные не являются достверным источником для исследования кредитного скоринга, пустые поля будут заполнены нулями.

В графе "ежемесячный доход" пропуски заполнены исходя из типа дохода клиента. Для каждого типа дохода подсчитана медиана, этим значением заполнены пропуски в данных.

### Замена типа данных

In [12]:
employed_error = []
try:
    data['days_employed'] = data['days_employed'].astype('int')
    data['total_income'] = data['total_income'].astype('int')
except:
    employed_error.append(data['days_employed'])
    print(employed_error)
    
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

"Общий трудовой стаж в днях" и "ежемесячный оклад" представлены в формате float, поэтому воспользуемся методом astype() и переведем в формат int для уменьшения объема файла и правильного представления данных.

### Обработка дубликатов

In [13]:
print('Обнаружено {:} полных дубликата(ов)'.format(data.duplicated().sum()))
data['education_lowercase'] = data['education'].str.lower()
data['education_lowercase'] = data['education_lowercase'].replace('неоконченное высшее','среднее')
data['education_lowercase'] = data['education_lowercase'].replace('ученая степень','высшее')
data['fam_status_lowercase'] = data['family_status'].str.lower()
data['fam_status_lowercase'] = data['fam_status_lowercase'].replace('гражданский брак','женат / замужем')
data['fam_status_lowercase'] = data['fam_status_lowercase'].replace(['в разводе','вдовец / вдова'],'не женат / не замужем')
data = data.drop_duplicates().reset_index(drop=True)

Обнаружено 54 полных дубликата(ов)


### Вывод

В столбцах "Образование" и "Семейное положение" выравнен регистр. В столбце образование заменены зачения "неоконченное высшее" на "среднее" и "ученая степень" на "высшее". В столбце семейное положение заменены зачения "гражданский брак" на "женат / замужем" и "в разводе", "вдовец / вдова" на "не женат / не замужем". 

Из датасета методом drop_duplicates() удалены 54 полных дубликата и сброшены индексы методом reset_index(). Возможная причина появление дубликатов - повторные заявки клиентов на получние кредита.

### Лемматизация

In [14]:
from pymystem3 import Mystem
m = Mystem()
from collections import Counter
def purpose_lem (lemm):
    lemmas = m.lemmatize(lemm)       
    if 'свадьба' in lemmas:
        return 'свадьба'
    if 'жилье' in lemmas:
        return 'жилье'
    if 'недвижимость' in lemmas:
        return 'недвижимость'
    if 'автомобиль' in lemmas:
        return 'автомобиль'
    if 'образование' in lemmas:
        return 'образование'
data['short_purpose'] = data['purpose'].apply(purpose_lem)

### Вывод

Для каждого индекса столбца purpose применяется функция purpose_lem, которая сначала разбивает поле на леммы, затем ищет указанные цели в леммах и, в случае удачи, присваивает обобщенную цель каждому полю нового столбца [short_purpose]

### Категоризация данных

In [15]:

data['income_quant'] = pd.qcut(data['total_income'], 5)


### Вывод

Поле "ежемесячный доход" с помощь квинтилей (quintile) разбиваем на пять равных групп и присваиваем уровни дохода новому столбцу "доход по квантилям"

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [16]:
child_grouped = data.groupby('children').agg({'debt': ['count', 'sum']})
child_grouped['child_conversion'] = child_grouped['debt']['sum'] / child_grouped['debt']['count']*100
display(child_grouped.sort_values(by = 'children'))

debt       child_conversion
          count   sum                 
children                              
0         14107  1063         7.535266
1          4856   445         9.163921
2          2052   194         9.454191
3           330    27         8.181818
4            41     4         9.756098
5             9     0         0.000000
20           76     8        10.526316

### Вывод

В данной выборке бросается отсутствие просроченных кредитов у семей с 5-ю детьми, но количество кредитов нет смысла анализировать из-за малой выборки. Подобная ситуация наблюдается и с кредитами, выданными клиентам с 3-мя детьми по отношению к аналогичному показателю в семьях с 2-мя детьми - одна выборка почти в 10 раз больше другой.
В целом бездетные возвращают средства в срок чаще (на 2%).

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [17]:
fam_grouped = data.groupby('fam_status_lowercase').agg({'debt': ['count', 'sum']})
fam_grouped['fam_conversion'] = fam_grouped['debt']['sum'] / fam_grouped['debt']['count']*100
display(fam_grouped.sort_values(by='fam_conversion', ascending=False))

debt       fam_conversion
                       count   sum               
fam_status_lowercase                             
не женат / не замужем   4964   422       8.501209
женат / замужем        16507  1319       7.990549

### Вывод

Количество непогашенных в срок кредитов у одиноких на 0,5% выше.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [18]:
income_grouped = data.groupby('income_quant').agg({'debt': ['count', 'sum']})
income_grouped['income_conversion'] = income_grouped['debt']['sum'] / income_grouped['debt']['count']*100
display(income_grouped.sort_values(by = 'income_quant'))

debt      income_conversion
                      count  sum                  
income_quant                                      
(20666.999, 98554.0]   4295  344          8.009313
(98554.0, 132123.0]    4294  361          8.407080
(132123.0, 161287.0]   4294  375          8.733116
(161287.0, 214565.0]   4294  361          8.407080
(214565.0, 2265604.0]  4294  300          6.986493

### Вывод

Использовав квантили для разбиения уровней дохода клиентов, можно сказать, что заемщики с наибольшим уровнем дохода (214 565 - 2 265 604) возвращают заемные средства чаще других категорий. У остальных процент примерно равный.

- Как разные цели кредита влияют на его возврат в срок?

In [19]:

data_pivot = pd.pivot_table(data, index=['short_purpose'], values='debt', aggfunc=['count','sum'])

data_pivot['purpose_conversion'] = data_pivot[('sum', 'debt')] / data_pivot[('count', 'debt')]*100
                                   
display(data_pivot.sort_values(by='purpose_conversion', ascending=False))

,count,sum,purpose_conversion
,debt,debt,
short_purpose,,,
автомобиль,4308,403,9.354689
образование,4014,370,9.217738
свадьба,2335,186,7.965739
недвижимость,6353,474,7.461042
жилье,4461,308,6.904282


### Вывод

Кредиты на покупку автомобиля и на образование реже возвращают в срок, чем все остальные. Жилье в приоритете.

### Шаг 4. Общий вывод

В ходе работы над полученными данными было сделано слудующее: 

 - обнаружены пропущенные значения в данных о ежемесячном доходе, которые были заполены медианой по каждому типу дохода в отдельности.
 - заменен с вещественного на целочисленный тип данных в данных о трудовом стаже в днях и ежемесячном окладе.
 - были удалены полные дубликаты данных
 - распределены цели кредита на группы при помощи лемматизации
 - с помощью квантилей уровни дохода коиентов были разбиты на пять равных частей.

По итогам исследования можно сделать следующие выводы:

1. приориет необходимо отдать выдаче кредитов семейным людям без детей, где цель указана -  жилье.
2. меньше выдавать разрешение на получение кредита семьям с детьми, где цель кредита - автомобиль.
3. кредиторы с наибольшим уровнем дохода (214 565 - 2 265 604) возвращают заемные средства чаще других категорий

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [X]  файл изучен;
- [X]  определены пропущенные значения;
- [X]  заполнены пропущенные значения;
- [X]  есть пояснение, какие пропущенные значения обнаружены;
- [X]  описаны возможные причины появления пропусков в данных;
- [X]  объяснено, по какому принципу заполнены пропуски;
- [X]  заменен вещественный тип данных на целочисленный;
- [X]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [X]  удалены дубликаты;
- [X]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [X]  описаны возможные причины появления дубликатов в данных;
- [X]  выделены леммы в значениях столбца с целями получения кредита;
- [X]  описан процесс лемматизации;
- [X]  данные категоризированы;
- [X]  есть объяснение принципа категоризации данных;
- [X]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [X]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [X]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [X]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [X]  в каждом этапе есть выводы;
- [X]  есть общий вывод.